In [5]:
import pandas as pd

# Specify the file path
file_path = 'ML-ResourceScheduler.xlsx'
# Read the Excel file
df = pd.read_excel(file_path)
# Display the DataFrame
print(df)


      BookingID  EstDuration    JobDate JobStartTime RequiredAssetType  \
0         32254          6.0 2024-01-02     06:00:00        25T Franna   
1         28490          2.0 2024-01-02     07:00:00        20T Franna   
2         31797          8.0 2024-01-02     07:00:00   60T All Terrain   
3         32259          2.0 2024-01-02     18:00:00    20T City Class   
4         32249          8.0 2024-01-03     06:30:00        40T Franna   
...         ...          ...        ...          ...               ...   
3992      39554          8.0 2024-07-03     21:00:00  130T All Terrain   
3993      39711          8.0 2024-07-04     06:00:00   60T All Terrain   
3994      36634          8.0 2024-07-04     07:00:00  130T All Terrain   
3995      39716          8.0 2024-07-04     07:00:00  100T All Terrain   
3996      39831          4.0 2024-07-04     07:00:00   Table Top Rigid   

                                       Client  \
0                   InfraBuild Steel Rod Mill   
1            

In [6]:
import pandas as pd

# Specify the file path
file_path = 'ML-ResourceScheduler.xlsx'
# read both sheets worker and asset allocation history 
dfs = pd.read_excel(file_path, sheet_name=['Worker Allocation History', 'Asset Allocation History'])
worker_history=dfs['Worker Allocation History']
asset_history=dfs['Asset Allocation History']


In [7]:
unique_names = worker_history['WorkerSystemID'].unique().tolist() #  the list of unique names of workers 
unique_type = worker_history['WorkerTypeId'].unique().tolist()  #  the list of unique worker type id of workers 


print(unique_names) 
print(unique_type)

[302, 422, 313, 409, 296, 391, 376, 415, 425, 372, 375, 420, 364, 365, 350, 373, 426, 423, 307, 366, 424, 303, 345, 360, 297, 418, 380, 427, 335, 321, 408, 411, 379, 394, 382, 398, 324, 318, 330, 421, 299, 325, 305, 430, 337, 429, 419, 362, 322, 309, 306, 311, 314, 326, 316, 432, 393, 301, 300, 433, 434, 320, 315, 357, 331, 377, 319, 390, 323, 310, 388, 304, 402, 385, 413, 403, 383, 308, 406, 389]
[1, 7, 2, 3, 9, 5, 0]


In [8]:
import pandas as pd

# Assuming asset_history and worker_history are already loaded DataFrames

# List of unique Asset IDs
unique_assets = asset_history['AssetID'].unique().tolist()

# Function to initialize a list with zeros
def initialize_zero_list(length):
    return [0] * length

# Initialize an empty DataFrame to store the results and columns to it 
df = pd.DataFrame(columns=['WorkerSystemID'] + unique_assets + unique_type[:-1])

# List of unique Worker IDs
unique_names = worker_history['WorkerSystemID'].unique().tolist()

# Iterate over each unique worker to get his experience and what kind of vehicle he have drived
for i in unique_names:
    filtered_df = worker_history[worker_history['WorkerSystemID'] == i] # get all data related to this id
    
    # Extract BookingID and WorkerTypeId
    booking_ids = filtered_df['BookingID'].tolist()
    worker_type_ids = filtered_df['WorkerTypeId'].tolist()
    
    # Initialize worker_rating list with zeros same as number of data in a row for every person to initialize
    worker_rating = initialize_zero_list(1 + len(unique_assets) + len(unique_type[:-1]))
    worker_rating[0] = i
    
    # Iterate over both booking_ids and worker_type_ids simultaneously
    for item1, item2 in zip(booking_ids, worker_type_ids):
        if item2 == 0:
            filtered_asset_df = asset_history[asset_history['BookingID'] == item1]
            asset_id = filtered_asset_df['AssetID'].tolist()
            if asset_id:
                try:
                    index = unique_assets.index(asset_id[0])
                    worker_rating[index + 1] += 1  #populate with data on the same index as the  asset id is if it is related to driving 
                except ValueError:
                    print(f"Asset ID {asset_id[0]} not in the list")
        elif item2 == 1:                  # all working categories except driving an asset are populated from here
            worker_rating[-6] += 1
        elif item2 == 2:
            worker_rating[-4] += 1
        elif item2 == 3:
            worker_rating[-3] += 1
        elif item2 == 5:
            worker_rating[-1] += 1
        elif item2 == 7:
            worker_rating[-5] += 1
        elif item2 == 9:
            worker_rating[-2] += 1
    
    # Append the worker_rating to the DataFrame
    df.loc[len(df)] = worker_rating

# Display the resulting DataFrame
print(df.head())



   WorkerSystemID  CC201  603  FC257  FC259  FC41  FC42  FC21  FC256  FC254  \
0             302      0    0      0      4     0     0     0      1      4   
1             422      0    0      0      0     2     4     0      4      0   
2             313     71   12      2      3     1     2     1      0      0   
3             409      0    0      0      0     0     0     0      0      0   
4             296      0    0      0      0     0     0     0      0      0   

   ...  250B  120A  LR01  FC261    1  7  2  3  9  5  
0  ...     0     0     0      0  105  0  0  0  0  0  
1  ...     0     0     0      0  117  1  0  0  0  1  
2  ...     0     0     0      1    8  0  0  0  0  0  
3  ...     0     0     0      0   95  2  3  0  0  0  
4  ...     0     0     0      0  100  0  0  0  0  0  

[5 rows x 58 columns]


In [9]:
df = pd.DataFrame(df)
df
#df.to_csv('modified_dataframe.csv', index=False)

,WorkerSystemID,CC201,603,FC257,FC259,FC41,FC42,FC21,FC256,FC254,...,250B,120A,LR01,FC261,1,7,2,3,9,5
0,302,0,0,0,4,0,0,0,1,4,...,0,0,0,0,105,0,0,0,0,0
1,422,0,0,0,0,2,4,0,4,0,...,0,0,0,0,117,1,0,0,0,1
2,313,71,12,2,3,1,2,1,0,0,...,0,0,0,1,8,0,0,0,0,0
3,409,0,0,0,0,0,0,0,0,0,...,0,0,0,0,95,2,3,0,0,0
4,296,0,0,0,0,0,0,0,0,0,...,0,0,0,0,100,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,403,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
76,383,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
77,308,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
78,406,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
new_column_list = unique_assets + unique_type[:-1]  # Replace with your actual list
corresponding_numbers = list(range(1, len(new_column_list) + 1))
print(corresponding_numbers,new_column_list)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57] ['CC201', 603, 'FC257', 'FC259', 'FC41', 'FC42', 'FC21', 'FC256', 'FC254', 552, 250, 403, 'FC258', 102, 'FC40', 601, 'XHire Slew', 151, 602, 120, 'FC260', 103, 'XHIRE Franna', 402, 'MC31', 'FC253', 'FC255', 'T7 - STEP DECK', 'Labour', 131, 'FC22', 'FC251', 303, 'XHire Semi', 'Ancilliary Eqpt', 604, 'MR01', 'T8 - FLAT TOP', '131A', '250A', 'P1', '103A', '151A', '102B', 'HR01', '131B', '250C', '250B', '120A', 'LR01', 'FC261', 1, 7, 2, 3, 9, 5]


In [11]:
# Transpose the dataframe  created above to train properly 
transposed_df = df.T
transposed_df.columns =transposed_df.iloc[0]
transposed_df = transposed_df[1:]

new_column_list = unique_assets + unique_type[:-1]  #this column is for asset ids and works
corresponding_numbers = list(range(1, len(new_column_list) + 1)) # initialised a list equal to it because model can not taake string input so every asset id represents a number 

transposed_df.insert(0, 'AssetID',corresponding_numbers)
transposed_df.to_csv('transposed_dataframe.csv', index=False)

print(transposed_df.head())

WorkerSystemID  AssetID  302  422  313  409  296  391  376  415  425  ...  \
CC201                 1    0    0   71    0    0    0    0    0    3  ...   
603                   2    0    0   12    0    0    0    0    0    4  ...   
FC257                 3    0    0    2    0    0   27    0    0    1  ...   
FC259                 4    4    0    3    0    0    0    0    0    0  ...   
FC41                  5    0    2    1    0    0    4    0    0    3  ...   

WorkerSystemID  388  304  402  385  413  403  383  308  406  389  
CC201             0    0    0    0    0    0    0    0    0    0  
603               0    4    0    0    0    0    0    0    0    0  
FC257             0    0    0    0    0    0    0    0    0    0  
FC259             0    1    0    0    0    0    0    0    0    0  
FC41              0    0    0    0    0    0    0    0    0    0  

[5 rows x 81 columns]


In [12]:
worker_ids = transposed_df['AssetID']
df_f = transposed_df.drop('AssetID', axis=1) #drop worker id from data to use it separately for training 

In [13]:
df_f.columns = df_f.columns.astype(str)
df_f =df_f.apply(pd.to_numeric)   #convert all columns to string for training